## **gold-yahoo-financial-statements**

This notebook builds the **`yahoo_financial_statements`** Delta table by combining income statement, cash flow, balance sheet, and company info data from the Silver layer. It unifies financial records into a single dataset and performs an upsert to keep the table updated. Error handling ensures smooth processing and traceability of issues.


#### **Importing necessary libraries**

In [6]:
from pyspark.sql.functions import *
from delta.tables import DeltaTable
import datetime
import traceback

StatementMeta(, a0e19049-e6f4-4237-b761-f44dbc410b57, 40, Finished, Available, Finished)

#### **Variables**

In [7]:
layer_name = "silver"
table_name = "yahoo_financial_statements"

StatementMeta(, a0e19049-e6f4-4237-b761-f44dbc410b57, 41, Finished, Available, Finished)

#### **Step 2: Reading the Stock Financial Statements and Information Data**

In [8]:
icm_stmt_df = spark.read.format('delta').load(f"Files/{layer_name}/comp_incm_stmt_data/")

cash_flow_df = spark.read.format('delta').load(f"Files/{layer_name}/comp_cash_flow_data/")

bal_sheet_df = spark.read.format('delta').load(f"Files/{layer_name}/comp_bal_sheet_data/")

com_info_df = spark.read.format('delta').load(f"Files/{layer_name}/comp_info_data/")

StatementMeta(, a0e19049-e6f4-4237-b761-f44dbc410b57, 42, Finished, Available, Finished)

#### **Step 3: Union of Balance Sheet, Income Statement, and Cash Flow DataFrames**

In [ ]:
icm_df = icm_stmt_df.select(col('Breakdown'), col('Company')).distinct()

cf_df = cash_flow_df.select(col('Breakdown'), col('Company')).distinct()

bal_sh_df = bal_sheet_df.select(col('Breakdown'), col('Company')).distinct()

fin_stmnt_df = icm_df.union(cf_df).union(bal_sh_df).distinct()


StatementMeta(, a0e19049-e6f4-4237-b761-f44dbc410b57, 43, Finished, Available, Finished)

#### **Step 4: Joining Financial Statements and Company Information**

In [ ]:
output_df = (
    broadcast(fin_stmnt_df)
    .join(
        icm_stmt_df.alias('inc_stmt'),
        (fin_stmnt_df["Breakdown"] == col("inc_stmt.Breakdown")) &
        (fin_stmnt_df["Company"] == col("inc_stmt.Company")),
        "left"
    )
    .join(
        cash_flow_df.alias('cash_flow'),
        (fin_stmnt_df["Breakdown"] == col("cash_flow.Breakdown")) &
        (fin_stmnt_df["Company"] == col("cash_flow.Company")),
        "left"
    )
    .join(
        bal_sheet_df.alias('bal_sht'),
        (fin_stmnt_df["Breakdown"] == col("bal_sht.Breakdown")) &
        (fin_stmnt_df["Company"] == col("bal_sht.Company")),
        "left"
    )
    .join(
        com_info_df,
        fin_stmnt_df["Company"] == com_info_df["company_name"],
        "left"
    )
    .select(
        fin_stmnt_df['Company'],
        com_info_df['short_name'].alias('company_name'),
        com_info_df['company_image'],
        com_info_df['industry'],
        com_info_df['sector'],
        fin_stmnt_df["Breakdown"],
        col("inc_stmt.amount").alias("income_statement"),
        col("inc_stmt.Date").alias("income_statement_date"),
        col("cash_flow.amount").alias("cash_flow"),
        col("cash_flow.Date").alias("cash_flow_date"),
        col('bal_sht.amount').alias('balance_sheet'),
        col("bal_sht.Date").alias("balance_sheet_date")
    )
)


StatementMeta(, a0e19049-e6f4-4237-b761-f44dbc410b57, 44, Finished, Available, Finished)

#### **Step 5: Storing the transformed data into a Delta table in the Gold layer**

In [10]:
try:
    if DeltaTable.isDeltaTable(spark, table_name):
        delta_table = DeltaTable.forPath(spark, table_name)
               
        delta_table.alias("target").merge(
            output_df.alias("source"),
                (
                    (col("target.company_name") == col("source.company_name")) &
                    (col("target.Breakdown") == col("source.Breakdown")) &
                    (col("target.income_statement_date") == col("source.income_statement_date")) &
                    (col("target.cash_flow_date") == col("source.cash_flow_date")) &
                    (col("target.balance_sheet_date") == col("source.balance_sheet_date"))
                )
            ).whenMatchedUpdateAll() \
             .whenNotMatchedInsertAll() \
             .execute()

    else:
        output_df.write.format("delta").mode("overwrite").saveAsTable(table_name)

except Exception as e:
    print("An error occurred during the processing:")
    traceback.print_exc()

StatementMeta(, a0e19049-e6f4-4237-b761-f44dbc410b57, 45, Finished, Available, Finished)